In [8]:
import pandas as pd
from time import sleep
from selenium import webdriver
from parsel import Selector
import pickle

## driver for posts

In [569]:
driver = webdriver.Chrome('D://ChromeDriver//Nouveau dossier//chromedriver.exe')
driver.get('https://www.facebook.com/')
sleep(1)
username = driver.find_element_by_name("email")
username.send_keys(""" adresse email """)
sleep(1)
password = driver.find_element_by_name('pass')
password.send_keys(""" mot de passe """)
sleep(1)
sign_in_button = driver.find_element_by_id('u_0_b')
sign_in_button.click()
sleep(1)

## driver for info

In [570]:
driver2 = webdriver.Chrome('D://ChromeDriver//Nouveau dossier//chromedriver.exe')
driver2.get('https://www.facebook.com/')
sleep(1)
username = driver2.find_element_by_name("email")
username.send_keys(""" adresse email """)
sleep(1)
password = driver2.find_element_by_name('pass')
password.send_keys(""" mot de passe """)
sleep(1)
sign_in_button = driver2.find_element_by_id('u_0_b')
sign_in_button.click()
sleep(1)

### get post id

In [ ]:
post_id=[]
pid=""
newPage=""
for p in range(0,20):
    if(p==0):
        url="https://mbasic.facebook.com/SocieteTunisiennedeBanquePageOfficielle/"
    if(p>0):
        url="https://mbasic.facebook.com/"+newPage
    driver.get(url)
    sleep(1)
    sel = Selector(text = driver.page_source)
    for i in range(1,5):
        article=""
        article=sel.xpath("/html/body/div/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div/section/article["+str(i)+"]").extract_first()
        article=article.split()
        ch=""
        ch=article[4]
        for p in range(65,81):
            pid+=ch[p]
        post_id.append(pid)
        pid=""
    newPage=""
    newPage=sel.xpath("/html/body/div/div/div[2]/div/div[1]/div[2]/div[2]/div[2]/a/@href").extract_first()
    print(newPage)


## Scrapping

In [641]:
final=[]
for post in range(len(post_id)):
    print(post)
    com=[]
    cont=[]
    infolist=[]
    datep=''
    for i in range(0,5):
        if(i==0):
            url='https://mbasic.facebook.com/SocieteTunisiennedeBanquePageOfficielle/posts/'+str(post_id[post])
            
            driver.get(url)
            sleep(5)
            sel = Selector(text = driver.page_source)
            date=sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[1]/div/footer/div[1]/abbr/text()').extract_first()
            datep=date
            
            content=driver.find_element_by_id('u_0_0').text
                
            print(content)
            cont.append(content)
        if(i>0):
            if(i==1):
                url=sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div[11]/a/@href').extract_first()
                url='https://mbasic.facebook.com/'+str(url)
                driver.get(url)
                sleep(5)
                sel = Selector(text = driver.page_source)
            else:
                p=i*10
                #url='https://mbasic.facebook.com/story.php?story_fbid='+str(post_id[post])+'&id=421001574752082&p='+str(p)+'&av=1595389103&eav=AfYxLjkjqRtRODj_7RamgHlhIedpgybZB0rLEsBS025-xeSQUbSrntYHTcoxIzsKNAI'
                url=sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div[12]/a/@href').extract_first()
                print(sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div[12]/a/@href').extract_first())
                #print(url)
                url='https://mbasic.facebook.com/'+str(url)
                print(url)
                driver.get(url)
                sleep(5)
                sel = Selector(text = driver.page_source)
        
        for j in range(1,11):
            info=dict()
            name=sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(j)+']/div/h3/a/text()').extract_first()
            #comments = sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(j)+']/div/div[1]/text()').extract_first()
            #com.append(comments)
            profile=sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(j)+']/div/h3/a/@href').extract_first()
            print(profile)
            urlprofile='https://mbasic.facebook.com/'+str(profile)
            driver2.get(urlprofile)
            sleep(5)
            print(url)
            print(urlprofile)
            sel2 = Selector(text = driver2.page_source)
            work = driver2.find_elements_by_id('work')
            education = driver2.find_elements_by_id('education')
            living = driver2.find_elements_by_id('living')
            contact = driver2.find_elements_by_id('contact-info')
            basic = driver2.find_elements_by_id('basic-info')
            if(sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(j)+']/div/div[1]/text()').extract_first() is None):
                break
            else:
                commentaire=sel.xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(j)+']/div/div[1]/text()').extract_first()
            #com.append(comments)
            worklist=[]
            educationlist=[]
            livinglist=[]
            contactlist=[]
            basiclist=[]
            for e in work:
                worklist.append(e.text)
            for e in education:
                educationlist.append(e.text)
            for e in living:
                livinglist.append(e.text)
            for e in contact:
                contactlist.append(e.text)
            for e in basic:
                basiclist.append(e.text)
            info['name']=name
            info['work']=worklist
            info['education']=educationlist
            info['living']=livinglist
            info['contact']=contactlist
            info['basic']=basiclist
            info['comment']=commentaire
            infolist.append(info)

    #### Remove empty fields
    com = [x for x in com if x != None]
    cont = [x for x in cont if x != None]
    ##### Remove duplication 
    cont = list(dict.fromkeys(cont))
    com=list(dict.fromkeys(com))
    ######
    
    ######## concat content 
    ch=""
    for i in range(len(cont)):
        ch+=cont[i]+" "
    postcom=[[ch,com,datep,infolist]]
    df=pd.DataFrame(postcom,columns=["content","comments","date","info"])
    with open('datas/data'+str(post), 'wb') as fp:
        #numfile='infolist'+str(post)
        pickle.dump(df, fp)
    #final.append(postcom)
    #df=pd.DataFrame(postcom,columns=["content","comments","date","info"])
    #df.to_csv(r'data'+str(post)+'.csv', index = False)


0
View Edit History
STB Bank الشركة التونسية للبنك
✅ STB NEO BY STB: Transformation to a Bank 4.0
💬 It was in an architectural jewelry of Tunisia, the Palais du Baron d ' Erlanger still called Ennejma Ezzahra, that the inauguration of the first edition of a national project called ′′ Neo By STB ′′ was held yesterday order transformation to a bank 4.0.
👌 ′′ Innovation is not just technological, innovation is first managerial says an expert expert in digital transformation, calling for a deep and collegial reflection, bringing together all the actors of a wide but still very very well-known ecosystem. little organized.
#NEOBY #STBLaBanqueEXponentielle #STBBANK #Neo
Translated from French
July 10 at 5:07 PM · Public
Like Page · Save · More
/rajamansour.zaabar?rc=p&__tn__=R
https://mbasic.facebook.com/SocieteTunisiennedeBanquePageOfficielle/posts/1413029672215929
https://mbasic.facebook.com//rajamansour.zaabar?rc=p&__tn__=R
/naouel.rekik.3?rc=p&__tn__=R
https://mbasic.facebook.com/SocieteT

### Rassember les données

In [9]:
dataf=pd.DataFrame(columns=["content","comments","date","info"])
for n in range(0,5):
    with open ('data'+str(n), 'rb') as fp:
        namefile = pickle.load(fp)
        dataf=dataf.append(namefile,ignore_index=True)
        

### Remove empty fields from dictionary

In [10]:
newinfo=dict()
newinfolist=[]
for j in range(len(dataf['info'])):
    newinfolist=[]
    for i in range(len(dataf['info'][j])):
        if(dataf['info'][j][i]['comment'] is not None):
            newinfo=dict()
            newinfo['name']=dataf['info'][j][i]['name']
            newinfo['work']=dataf['info'][j][i]['work']
            newinfo['education']=dataf['info'][j][i]['education']
            newinfo['living']=dataf['info'][j][i]['living']
            newinfo['contact']=dataf['info'][j][i]['contact']
            newinfo['basic']=dataf['info'][j][i]['basic']
            newinfo['comment']=dataf['info'][j][i]['comment']
            newinfolist.append(newinfo)
    dataf['info'][j]=newinfolist        

## Collect reactions for posts

In [585]:
good=""
bad=""
like=""
love=""
haha=""
care=""
wow=""
sad=""
angry=""
goodbad=[]
reacts=[]
for pid in range(len(post_id)):
    good=""
    bad=""
    url='https://mbasic.facebook.com/ufi/reaction/profile/browser/fetch/?limit=10&total_count=402&ft_ent_identifier='+str(post_id[pid])
    driver.get(url)
    sleep(5)
    sel = Selector(text = driver.page_source)
    for i in range(1,8):
        path=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/img/@alt').extract_first()
        if(path=='Like'):
            like=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
        if(path=='Love'):
            love=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
        if(path=='Care'):
            care=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
        if(path=='Haha'):
            haha=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
        if(path=='Sad')  :
            sad=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
        if(path=='Wow'):
            wow=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
        if(path=='Angry'):
            angry=sel.xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a['+str(i)+']/span/text()').extract_first()
    if(like is not None):
        if('.' in str(like)):
            like=str(like).replace('.','')
            like=str(like).replace('K','00')
            like=int(like)
        if('K' in str(like)):
            like=str(like).replace('K','000')
            like=int(like)
    if(like is None):
        like=0
    if(love is not None):
        if('.' in str(love)):
            love=str(love).replace('.','')
            love=str(love).replace('K','00')
            love=int(love)
        if('K' in str(love)):
            love=str(love).replace('K','000')
            love=int(love)
    if(love is None):
        love=0
    
    if(care ==''):
        care=0
    if(care is not None):
        if('.' in str(care)):
            care=str(care).replace('.','')
            care=str(care).replace('K','00')
            care=int(care)
        if('K' in str(care)):
            care=str(care).replace('K','000')
            care=int(care)
    if(haha is not None):
        if('.' in str(haha)):
            haha=str(haha).replace('.','')
            haha=str(haha).replace('K','00')        
            haha=int(haha)
        if('K' in str(haha)):
            haha=str(haha).replace('K','000')        
            haha=int(haha)
    if(haha == ''):
        haha=0
    if(angry is not None):
        if('.' in str(angry)):
            angry=str(angry).replace('.','')
            angry=str(angry).replace('K','00')
            angry=int(angry)
        if('K' in str(angry)):
            angry=str(angry).replace('K','000')
            angry=int(angry)
    if(angry ==''):
        angry=0
    if(sad is not None):
        if('.' in str(sad)):
            sad=str(sad).replace('.','')
            sad=str(sad).replace('K','00')
            sad=int(sad)
        if('K' in str(sad)):
            sad=str(sad).replace('K','000')
            sad=int(sad)
    if(sad ==''):
        sad=0
    if(wow is not None):
        if('.' in str(wow)):
            wow=str(wow).replace('.','')
            wow=str(wow).replace('K','00')
            wow=int(wow)
        if('K' in str(wow)):
            wow=str(wow).replace('K','000')
            wow=int(wow)
    if(wow ==''):
        wow=0
    #print(like+' '+love+' '+care)
    good=int(like)+int(love)+int(care)
    #good=like+love+care
    #print(like)
    bad=int(haha)+int(angry)+int(sad)+int(wow)
    goodbad=[[good,bad]]
    dfLikes=pd.DataFrame(goodbad,columns=["good","bad"])
    dfLikes.to_csv(r'likes/likes'+str(pid)+'.csv', index = False)

## Cleaning Posts

In [11]:
for i in range(len(dataf['content'])):
    dataf['content'][i]=dataf['content'][i].replace('View Edit History\nSTB Bank الشركة التونسية للبنك\n','')
    dataf['content'][i]=dataf['content'][i].replace('Public\nLike Page · Save · More ','')
    dataf['content'][i]=dataf['content'][i].replace('\nTranslated from\n','')
    dataf['content'][i]=dataf['content'][i].replace('\n','')
    dataf['content'][i]=dataf['content'][i].replace('See Translation','')
    dataf['content'][i]=dataf['content'][i].replace('PublicLike Page · Save · More ','')
    

## Collect likes

In [12]:
like0=pd.read_csv('likes0.csv')
like1=pd.read_csv('likes1.csv')
like2=pd.read_csv('likes2.csv')
like3=pd.read_csv('likes3.csv')
like4=pd.read_csv('likes4.csv')

In [13]:
likes=[like0,like1,like2,like3,like4]

In [14]:
react=pd.concat(likes,ignore_index=True)

## Concat dataframes

In [15]:
final = pd.concat([dataf, react], axis=1).reindex(dataf.index)

## Cleaning comments

In [16]:
final.drop(columns=['comments'],inplace=True)

In [17]:
final.content[0]=final.content[0].replace('View Edit History\nSTB Bank الشركة التونسية للبنك\n','')
final.content[0]=final.content[0].replace('\nTranslated from French\nJuly 10 at 5:07 PM · Public\nLike Page · Save · More ','')
final.content[0]=final.content[0].replace('\n','')

C:\Users\SELON\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\SELON\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\SELON\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This i

In [20]:
with open('datas/data', 'wb') as fp:
    pickle.dump(final, fp)